# Tutorial
*mescal* is a [Brightway](https://docs.brightway.dev/en/latest/)-powered Python package that helps you to integrate Life-Cycle Assessment (LCA) in your energy system model

## Set up

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

In [ ]:
ecoinvent_version = '3.10.1' # choose the ecoinvent version you wish to use
esm_location = 'CH' # choose the version of energyscope for which you want to generate metrics
esm_year = 2020 # choose the year of the energyscope snapshot model or transition pathway time-step
spatialized_database = False # set to True if you want to use your spatialized version of ecoinvent
regionalize_foregrounds = ['Operation'] # set to 'all' if you want to regionalize the foreground inventories of all types of LCI datasets
premise_iam = 'image' # choose the IAM to which the premise database is linked
premise_ssp_rcp = 'SSP2-Base' # choose the SSP/RCP scenario to which the premise database is linked

In [ ]:
# Set the name of your main LCI database (e.g., ecoinvent or premise database) here:
name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_{premise_iam}_{premise_ssp_rcp}_{esm_year}"
name_biosphere_db = 'ecoinvent-3.10.1-biosphere'
if spatialized_database:
    name_premise_db += ' regionalized'
    name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'
else:
    name_spatialized_biosphere_db = None

In [ ]:
# Set the name of the new database with the ESM results
esm_db_name = f'EnergyScope_{esm_location}_{esm_year}'

In [ ]:
# Main version of ecoinvent (without .1 if any)
ecoinvent_main_version = '.'.join(ecoinvent_version.split('.')[:2])

In [ ]:
# Set the list of LCIA methods for which you want indicators (they must be registered in your brightway project)
if spatialized_database:
    lcia_methods=[
        f'IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v{ecoinvent_main_version}', 
        f'IMPACT World+ Damage 2.1_regionalized for ecoinvent v{ecoinvent_main_version}',
    ]
else:
    lcia_methods=[
        f'IMPACT World+ Midpoint 2.1 for ecoinvent v{ecoinvent_main_version}', 
        f'IMPACT World+ Damage 2.1 for ecoinvent v{ecoinvent_main_version}', 
        f'IMPACT World+ Footprint 2.1 for ecoinvent v{ecoinvent_main_version}', 
    ]

**Note**: you can download IMPACT World+ methods in your brightway project following this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/download_impact_world_plus.ipynb). Regionalized versions of ecoinvent and IMPACT World+ methods can be downloaded from this [repository](https://github.com/matthieu-str/Regiopremise).

In [ ]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}') # put the name of your brightway project here

## Load databases

In [ ]:
premise_db = Database(name_premise_db, create_pickle=True)

In [ ]:
if regionalize_foregrounds & spatialized_database:
    spatialized_biosphere_db = Database(name_spatialized_biosphere_db)
else:
    spatialized_biosphere_db = None

## Your data
For mescal to understand the structure of your energy system model, you need to provide it with a set of dataframes.

### Mandatory dataframes

In [ ]:
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping.csv')
unit_conversion = pd.read_csv(f'../dev/energyscope_data/{esm_location}/unit_conversion.csv')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')

A mapping between the energy technologies and resources of the energy system model, and Life-Cycle Inventories datasets (LCI) from an LCI database (e.g., ecoinvent). The mapping should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', 'Resource' or 'Flow')  
- `Product`: the name of the product of the energy technology or resource in the LCI database
- `Activity`: the name of the activity of the energy technology or resource in the LCI database
- `Location`: the name of the location of the energy technology or resource in the LCI database
- `Unit`: (optional) the physical unit of the energy technology or resource in the LCI database
- `Database`: the name of the database in your brightway project

If you wish to change the version of ecoinvent used in your mapping file, you can follow this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/mapping_ecoinvent_version.ipynb). 

In [ ]:
mapping.head()

A set of unit conversion factors between the energy system model and the LCI database. The conversion factors should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy system model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', 'Resource', or 'Other'). Other types can be added to fit your needs. The 'Other' category is meant for the unit conversion that are not specific to a technology or resource, but rather a generic type of product, e.g., conversion from kilogram to kilowatt hour for natural gas.
- `Value`: the numerical value of the conversion factor that will multiply the impact scores. It actually denotes the conversion from Impact / LCA unit to Impact / ESM unit. 
- `ESM`: the unit of the energy technology or resource in the ESM
- `LCA`: the target unit of the energy technology or resource in the LCA database
- `Assumptions & Sources` (optional): additional information on the conversion factor.

The `LCA` and `ESM` columns should respect the ecoinvent naming convention. You may use the `ecoinvent_unit_convention` function to convert the unit naming convention you've used to the one of ecoinvent.

In [ ]:
unit_conversion.head()

A mapping between the energy system model flows and CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Flow`: the name of the flow in the ESM
- `Description`: (optional) the description of the product
- `CPC`: the list of names of corresponding CPC categories

In [ ]:
mapping_esm_flows_to_CPC.head()

The input and output flows of energy technologies. For a given technology, the inputs and outputs should be given with the same unit. Also, the main output flow should have 1 as an amount, i.e., all other flows as scaled with respect to the main output. It should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy system model
- `Flow`: the name of the input or output flow
- `Amount`: the numerical value of the flow (negative if input, positive if output)  

In [ ]:
model.head()

### Optional dataframes

In [ ]:
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv') 
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
efficiency = pd.read_csv(f'../dev/energyscope_data/{esm_location}/efficiency.csv')
mapping_product_to_CPC = pd.read_csv('../mescal/data/mapping_new_products_to_CPC.csv')
impact_abbrev = pd.read_csv('../dev/lcia/impact_abbrev.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
results_from_esm = pd.read_csv(f'../dev/energyscope_data/{esm_location}/results_ES.csv')

A set of composition of technologies, i.e., if one technology or resource in the energy model should be represented by a combination of LCI datasets. The composition should be provided in a dataframe with the following columns:
- `Name`: the name of the main energy technology or resource in the energy model 
- `Components`: the list of names of subcomponents

In [ ]:
technology_compositions.head()

A set of technologies with specific requirements. For instance, this stands for energy technologies without a construction phase, mobility technologies (if mismatch fuel in the LCI dataset), bio-processes (if mismatch fuel in the LCI dataset), etc. The requirements should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Specifics`: the type of requirement. Can be '**Background search**' (i.e., double-counting removal is run n levels in the background, n being defined in Amount), '**Decommissioning**' (i.e., the technology has a decommissioning LCI dataset outside its infrastructure LCI dataset), '**Mobility**' (i.e., EUD types for which associated technologies are characterized as a mobility mean, to further add a FUEL layer), '**No background search**' (i.e., double-counting removal is not applied beyond the activity direct exchanges), '**No double-counting removal**' (i.e., the double-counting removal step is skipped), '**No construction**' (i.e., the technology has no infrastructure LCI dataset), '**Process**' (i.e., the technology is characterized as an industrial bio-process, to further add a PROCESS_FUEL layer), '**DAC**' (for *premise* DAC technologies, to change the biogenic carbon flow into a fossil one), '**Biofuel**' (i.e., adapt direct emissions to the biofuel input: partially change fossil carbon emissions into biogenic ones), '**Import/Export**' (these activities will keep their original locations and will not be regionalized), '**Carbon flows**' (the direct carbon emissions of these activities will be multiplied by a factor), '**Add CC**' (add a carbon capture process to a technology, and modifies its direct carbon dioxide emissions according to the capture efficiency), '**Add CO2 (flow_type)**' (add a resource/emission flow of CO2 to an activity, flow_type can be 'fossil', 'non-fossil', 'from soil or biomass stock', 'in air', or 'non-fossil, resource correction').

In [ ]:
technology_specifics.head()

Energy technologies lifetimes in the ESM and the LCI database. For composition of technologies (if any), the main technology should not have a LCA lifetime (no LCI dataset is associated to it), while the sub-components should not have an ESM lifetime (they do not have a proper technology in the ESM). The lifetimes should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy system model
- `ESM`: the numerical value of the lifetime in the energy system model (if relevant)
- `LCA`: the numerical value of the lifetime in the LCI database (if relevant)
- `Comment`: (optional) additional information the `Amount` computation (if relevant)

In [ ]:
lifetime.head()

If you want to account for the possible efficiency differences between the ESM and the LCI datasets, you can provide a set of (technologies, flow) couples for which efficiency differences will be corrected by the scaling the elementary flows. Taking the example of the diesel car, the couple should be something like ('CAR_DIESEL', 'DIESEL'), the flow being the fuel of the technology responsible for direct emissions. Relevant biosphere flows will be the ratio between the LCA and ESM efficiencies. The (technology, flow) couples should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Flow`: the name of the flow in the ESM
- `Comment`: (optional) additional information on the set of `Flows`

In [ ]:
efficiency.head()

In case you have a LCI database (partially) without CPC categories (which are necessary for the double-counting check), you can provide a mapping between the products and activities in the LCI database and the CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Name`: the full or partial name of the product or activity in the LCI database
- `CPC`: the number and name of the corresponding CPC category
- `Search type`: can be 'equals' if the `Name` entry is an exactly the name to look for, or 'contains' if it is contained in the full name
- `Where`: can be 'Product' or 'Activity', whether the `Name` entry is meant for products or activities

In [ ]:
mapping_product_to_CPC.head()

An abbreviation scheme for the impact categories you aim to work with, to ease the readability in the ESM. The one of IMPACT World+ is available in this [csv file](https://github.com/matthieu-str/mescal/blob/master/dev/lcia/impact_abbrev.csv). The abbreviations should be provided in a dataframe with the following columns:
- `Impact_category`: the name of the impact category, expressed as a tuple following brightway convention
- `Unit`: (optional) the unit of the impact category
- `Abbrev`: the abbreviation of the impact category
- `AoP`: the area of protection of the impact category. The normalization of indicators will be performed based of AoPs. If you do not have AoPs (e.g., midpoint-level indicators), set as many AoPs as you have impact categories.

In [ ]:
impact_abbrev.head()

In case you want to remove some energy technologies from energy layers in the results LCI datasets to be added to the LCI database, you can provide a list of technologies to remove. The list should be provided in a dataframe with the following columns:
- `Layers`: name of the layer(s). A layer is basically an energy vector, which is an output for some energy technologies and an input for some others. 
- `Technologies`: the name of the energy technology to remove from the layer(s)
- `Comment`: (optional) a comment on the removal

In [ ]:
technologies_to_remove_from_layers.head()

If you want to reformat your end-use types (e.g., output layer) in order to better fit the LCI database, you can provide a list of new end-use types. The list should be provided in a dataframe with the following columns:
- `Name`: name of technologies for which the end-use type should be changed
- `Search type`: whether the `Name` entry is an exactly the name to look for ('equals'), is contained in the full name ('contains'), or is the beginning of the full name ('startswith')
- `Old`: the current end-use type
- `New`: the new end-use type

In [ ]:
new_end_use_types.head()

If you want to inject the results of your ESM back in the LCI database, you should provide the results in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Production`: the annual production value of the energy technology in the ESM. All values should be provided with the same unit.
- `Capacity`: the installed capacity of the energy technology in the ESM. All values should be provided with the same unit.
- `Year`: the year of the ESM snapshot or transition pathway time-step.
- `Year_inst`: the year of installation of the energy technology. This is only required for `PathwayESM` if `operation_metrics_for_all_time_steps` is True.

In [ ]:
results_from_esm.head()

### Optional geographical data

In [ ]:
# sufficient match within ecoinvent
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC', 'CA']
elif esm_location == 'CH':
    accepted_locations = ['CH']
else:
    accepted_locations = ['GLO', 'RoW']

In [ ]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CAN', # Canada in IMAGE
        'CAZ', # Canada - Australia - New Zealand in REMIND
        'RNA', # North America
        'US', # United States
        'USA', # United States in REMIND and IMAGE
        'GLO', # Global average 
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH',
        'NEU',
        'EUR',
        'WEU',
        'RER', 
        'IAI Area, EU27 & EFTA',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

## Create a new database with additional CPC categories (optional)
In case you are working with a LCI database without CPC categories, you can create a new database with the CPC categories. The function `create_new_database_with_CPC_categories` takes as input the database with missing CPC categories, the name of the new database, and a mapping between the products and activities in the LCI database and the CPC categories. It creates a new database with the CPC categories. This step can take a few minutes depending on the size of the database.

In [ ]:
# If necessary, add missing CPC categories to the database
premise_db.add_CPC_categories()

## Initialize the ESM database

In [ ]:
mapping.Database = name_premise_db

In [ ]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    main_database=premise_db,
    esm_db_name=esm_db_name,
    biosphere_db_name=name_biosphere_db,
    esm_location=esm_location,

    # Optional inputs
    technology_compositions=technology_compositions,
    tech_specifics=technology_specifics,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    locations_ranking=my_ranking,
    spatialized_biosphere_db=spatialized_biosphere_db,
    results_path_file=f'results/energyscope_{esm_location}/{esm_year}/',
)

In [ ]:
esm.check_inputs()

## Add or replace the location column based on a user-defined ranking (optional)
Based on a user-defined ranking, the location column of the mapping dataframe can be updated. The function `change_location_mapping_file` takes as input the mapping dataframe, the user-defined ranking, and the base database. It returns the mapping dataframe with the location column updated.

In [ ]:
# Update mapping dataframe with better locations
esm.change_location_mapping_file()

In [ ]:
esm.mapping.head()

## Create ESM database after double-counting removal, efficiency harmonization, and lifetime harmonization

In [ ]:
esm.create_esm_database()

## Computing the LCA metrics

In [ ]:
R_long_direct_emissions, _, _ = esm.compute_impact_scores(
    methods=lcia_methods,
    assessment_type='direct emissions',
    overwrite=False,
)

In [ ]:
R_long_direct_emissions.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores_direct_emissions.csv', index=False)

In [ ]:
validation_direct_cf = esm._validation_direct_carbon_footprints(
    R_direct=R_long_direct_emissions,
    lcia_method_carbon_footprint=('IMPACT World+ Midpoint 2.1 for ecoinvent v3.10', 'Midpoint', 'Climate change, short term'),
    carbon_flow_in_esm='CO2_E',
)

In [ ]:
R_long, contrib_analysis_res, _ = esm.compute_impact_scores(
    methods=lcia_methods,
    contribution_analysis='emissions',
)

In [ ]:
R_long.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv', index=False)
contrib_analysis_res.to_csv(f'results/energyscope_{esm_location}/{esm_year}/contribution_analysis.csv', index=False)

In [ ]:
R_long.head()

In [ ]:
contrib_analysis_res.head()

## Convert the results in an AMPL format

In [ ]:
# To skip the previous steps
# R_long = pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv')
# R_long_direct_emissions = pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores_direct_emissions.csv')

In [ ]:
R_long.Value *= 1e6 # from FU / kW(h)-pkm-tkm to FU / GW(h)-Mpkm-Mtkm

In [ ]:
R_long_direct_emissions.Value *= 1e6 # from FU / kW(h)-pkm-tkm to FU / GW(h)-Mpkm-Mtkm

In [ ]:
# Tou can specify which specific LCIA methods or impact categories you wish to use using one of the three following options:
specific_lcia_methods = ['IMPACT World+ Footprint 2.1 for ecoinvent v3.10'] # selects the specific methods via their names
# specific_lcia_categories = ['Total ecosystem quality', 'Total human health']  # selects the specific impact categories via their names 
# specific_lcia_abbrev = ['TTHH', 'TTEQ', 'm_CCS'] # selects the specific impact categories via their abbreviations

In [ ]:
# Additional information that can be added at the beginning of the AMPL .mod and .dat files
metadata = {
    'ecoinvent_version': ecoinvent_version,
    'year': esm_year,
    'spatialized': spatialized_database,
    'regionalized': regionalize_foregrounds,
    'iam': premise_iam,
    'ssp_rcp': premise_ssp_rcp,
    'lcia_methods': lcia_methods,
}

### Normalize LCA indicators and create the .dat file

In [ ]:
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=specific_lcia_methods,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
)

In [ ]:
esm.normalize_lca_metrics(
    R=R_long_direct_emissions,
    mip_gap=1e-6,
    lcia_methods=lcia_methods,
    # specific_lcia_abbrev=specific_lcia_abbrev,
    assessment_type='direct emissions',
    max_per_cat=pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/res_lcia_max.csv'),
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
    file_name='techs_lcia_direct',
)

### Create the .mod file

In [ ]:
esm.generate_mod_file_ampl(
    lcia_methods=specific_lcia_methods,
    # specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
)

In [ ]:
esm.generate_mod_file_ampl(
    lcia_methods=lcia_methods,
    # specific_lcia_abbrev=specific_lcia_abbrev,
    assessment_type='direct emissions',
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
    metadata=metadata,
    file_name='objectives_direct',
)

## Integrate the ESM results back in the LCI database

In [ ]:
esm.create_new_database_with_esm_results(
    esm_results=results_from_esm,
    new_end_use_types=new_end_use_types,
    tech_to_remove_layers=technologies_to_remove_from_layers,
    write_database=True,
    remove_background_construction_flows=True,
    harmonize_with_esm=False,
)

In [ ]:
esm.connect_esm_results_to_database(create_new_db=True)